In [1]:
# Notebook for estimating a case-hospitalization rate, per age
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
plt.rcParams['lines.markersize'] = 10
plt.rcParams['lines.linewidth'] = 3
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))

import os
# import csv
import math


from datetime import date


saveFigures = True
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath

In [3]:
dfAdm = pd.read_excel('Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')

In [6]:
# dfCase = pd.read_csv(latestdir+'\\Cases_by_age.csv',delimiter=';')
dfCase = pd.DataFrame(columns=dfAdm.columns)
dfTest = pd.DataFrame(columns=dfAdm.columns)
dfPosP = pd.DataFrame(columns=dfAdm.columns)

In [8]:
# for d in curDates:
for k in range(0,len(curDates)):
    d = curDates[k]
    curIndex = dfAdm.index[k]

    thisDateStr = d.strftime('%Y-%m-%d')
    curDir = rootdir + '/SSI_data_' + thisDateStr 
    curFilePath = curDir + '/Cases_by_age.csv'
    curdf = pd.read_csv(curFilePath,delimiter=';',dtype=str)


    curdf['Antal_bekræftede_COVID-19'] = pd.to_numeric(curdf['Antal_bekræftede_COVID-19'].astype(str).apply(lambda x: x.replace('.','')))
    curdf['Antal_testede'] = pd.to_numeric(curdf['Antal_testede'].astype(str).apply(lambda x: x.replace('.','')))
    curdf['Procent_positive'] = pd.to_numeric(curdf['Procent_positive'].astype(str).apply(lambda x: x.replace(',','.')))

    dfCase.loc[curIndex] = curdf['Antal_bekræftede_COVID-19'].values
    dfTest.loc[curIndex] = curdf['Antal_testede'].values
    dfPosP.loc[curIndex] = curdf['Procent_positive'].values



In [14]:
dfCaseDiff = dfCase.diff().iloc[1:]
dfAdmDiff = dfAdm.diff().iloc[1:]
dfCaseDiff
dfAdmDiff

Aldersgruppe,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,I alt
17_11_2020,2,4,13,18,26,49,55,63,42,14,286
24_11_2020,8,1,16,19,28,49,50,59,70,11,311
01_12_2020,4,1,7,12,32,45,33,63,59,15,271
08_12_2020,6,7,23,32,41,61,67,110,80,21,448
15_12_2020,12,14,27,48,59,102,116,120,103,36,637
22_12_2020,22,15,42,70,95,143,152,236,185,54,1014
29_12_2020,15,12,48,50,98,148,170,285,212,61,1099
05_01_2021,5,14,37,58,72,121,161,211,224,73,976
12_01_2021,12,6,33,37,49,108,116,206,211,58,836
19_01_2021,12,3,15,32,46,87,97,167,152,57,668


In [62]:
caseAdmRate = np.divide(dfAdmDiff.iloc[1:].values,dfCaseDiff[:-1])
# caseAdmRate = np.divide(dfAdmDiff.iloc[2:].values,dfCaseDiff[:-2])
# dfCaseDiff.iloc[:-1]
# dfAdmDiff.iloc[1:]

In [142]:
# caseAdmRate.mean()

# allCols = caseAdmRate.columns[:-1]
allCols = caseAdmRate.columns

plotDates = curDates[2:]
# plotDates = curDates[3:]

fig,ax1 = plt.subplots()

finalEstimates = []

numToShow = len(allCols)-1
numToShow = len(allCols)-4
cmap = plt.cm.get_cmap('Dark2',numToShow)
for k in range(0,numToShow):
    curCol = allCols[k]

    ax1.plot(plotDates,100*caseAdmRate[curCol],'.:',markersize=3,linewidth=0.75,color=cmap(k))
    meanWidth = 4
    ax1.plot(rnTime(plotDates,meanWidth),rnMean(100*caseAdmRate[curCol],meanWidth),color=cmap(k),label=curCol)

    # Collect the final mean values
    curFinalMean = rnMean(100*caseAdmRate[curCol],meanWidth)[-1]
    finalEstimates.append(curFinalMean)

ax1.legend()
ax1.set_ylim([0,20])
# ax1.set_ylim([0,100])
# ax1.set_ylim(bottom=0)
ax1.set_xlim([plotDates[0],plotDates[-1]])

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

curFont = 18
ax1.set_ylabel('Indlæggelser per tilfælde ugen før [%]',fontsize=curFont)
plt.xticks(fontsize=curFont)
plt.yticks(fontsize=curFont)
plt.grid()

plt.tight_layout()

if saveFigures:
    plt.savefig('figs/case_hospitalization_per_time')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [144]:
# caseAdmRate.mean()

# allCols = caseAdmRate.columns[:-1]
allCols = caseAdmRate.columns

plotDates = curDates[2:]
# plotDates = curDates[3:]

# fig,ax1 = plt.subplots()

finalEstimates = []

numToShow = len(allCols)-1
# numToShow = len(allCols)-4
cmap = plt.cm.get_cmap('Dark2',numToShow)
for k in range(0,numToShow):
    curCol = allCols[k]

    # ax1.plot(plotDates,100*caseAdmRate[curCol],'.:',markersize=3,linewidth=0.75,color=cmap(k))
    meanWidth = 4
    # ax1.plot(rnTime(plotDates,meanWidth),rnMean(100*caseAdmRate[curCol],meanWidth),color=cmap(k),label=curCol)

    # Collect the final mean values
    curFinalMean = rnMean(100*caseAdmRate[curCol],meanWidth)[-1]
    finalEstimates.append(curFinalMean)

# ax1.legend()
# ax1.set_ylim([0,20])
# # ax1.set_ylim([0,100])
# # ax1.set_ylim(bottom=0)
# ax1.set_xlim([plotDates[0],plotDates[-1]])

# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

# curFont = 18
# ax1.set_ylabel('Indlæggelser per tilfælde ugen før [%]',fontsize=curFont)
# plt.xticks(fontsize=curFont)
# plt.yticks(fontsize=curFont)
# plt.grid()

# plt.tight_layout()

# if saveFigures:
#     plt.savefig('figs/case_hospitalization_per_time')

In [145]:
# print(allCols.values[:numToShow])
# print(finalEstimates)
print(f'Alderstrin:   {allCols.values[0]}: {finalEstimates[0]:.1f} %')
for k in range(1,numToShow):
    print(f'Alderstrin: {allCols.values[k]}: {finalEstimates[k]:.1f} %')

Alderstrin:   0-9: 0.9 %
Alderstrin: 10-19: 0.6 %
Alderstrin: 20-29: 2.0 %
Alderstrin: 30-39: 4.2 %
Alderstrin: 40-49: 4.9 %
Alderstrin: 50-59: 7.8 %
Alderstrin: 60-69: 12.6 %
Alderstrin: 70-79: 22.3 %
Alderstrin: 80-89: 30.6 %
Alderstrin: 90+: 38.0 %


In [146]:
plt.figure()
plt.bar(range(0,numToShow),finalEstimates,color='k')

for i,v in enumerate(finalEstimates):
    plt.text(i,v+0.5,f'{v:.1f} %',fontsize=14,va='center',ha='center')

curFont = 18
plt.yticks(fontsize=curFont-2)
plt.xticks(range(0,numToShow),labels=allCols.values[:-1],fontsize=curFont-2)
plt.ylabel('Indlæggelser per tilfælde ugen før [%]',fontsize=curFont)
plt.xlabel('Alderstrin',fontsize=curFont)

plt.title('4 ugers gennemsnit op til '+plotDates[-1].strftime('%d. %b'))
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/HospitalizationPerAge')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [129]:
# Calculation of size of age-group
dfAges = pd.read_excel('DKfolketal2021clean.xlsx')

In [158]:
DK_age = []
DK_age.append(dfAges.iloc[0:10].Antal.sum())
DK_age.append(dfAges.iloc[10:20].Antal.sum())
DK_age.append(dfAges.iloc[20:30].Antal.sum())
DK_age.append(dfAges.iloc[30:40].Antal.sum())
DK_age.append(dfAges.iloc[40:50].Antal.sum())
DK_age.append(dfAges.iloc[50:60].Antal.sum())
DK_age.append(dfAges.iloc[60:70].Antal.sum())
DK_age.append(dfAges.iloc[70:80].Antal.sum())
DK_age.append(dfAges.iloc[80:90].Antal.sum())
DK_age.append(dfAges.iloc[90:].Antal.sum())
DK_age = np.array(DK_age)
DK_age

array([610210, 680040, 778740, 696679, 748824, 801166, 667583, 574697,
       236648,  45458], dtype=int64)

In [187]:
# R0 = 2.5
# R0 = 2.5 * 1.5
R0 = 1.7
herdImmunityThreshold = 1-(1/R0)

vaccPerc = 0.010
immuPerc = 0.08
vaccPerc = 0.24
immuPerc = 0.08

vaccPercList = [0,0.008,0.10,0.131,0.18,0.21,0.52,0.93,0.95,0.94]
herdImmunityThresholdList = np.

missingToInfectPerc = herdImmunityThreshold - vaccPerc - immuPerc
print(missingToInfectPerc)
missingToInfect = np.multiply(missingToInfectPerc,DK_age)
print(missingToInfect)

missingToInfect



0.09176470588235293
[55995.74117647 62403.67058824 71460.84705882 63930.54352941
 68715.61411765 73518.76235294 61260.55764706 52736.90117647
 21715.93411765  4171.44      ]


In [188]:
toAdm = np.round(np.multiply(missingToInfect,np.multiply(finalEstimates,0.01)))
print(allCols.values[:-1])
print(toAdm)
toAdm[0:6].sum()

['0-9' '10-19' '20-29' '30-39' '40-49' '50-59' '60-69' '70-79' '80-89'
 '90+']
[  499.   343.  1447.  2683.  3390.  5745.  7724. 11752.  6645.  1587.]


14107.0